In [ ]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore

In [ ]:
project_id = "hacklyticsproject-451705"
location ="us"
processor_display_name = "hacklyticsprocessor"
processor_type = 

In [ ]:
def fetch_processor_types_sample(project_id: str, location: str) -> None:
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    # e.g.: projects/project_id/locations/location
    parent = client.common_location_path(project_id, location)

    # Fetch all processor types
    response = client.fetch_processor_types(parent=parent)

    print("Processor types:")
    # Print the available processor types
    for processor_type in response.processor_types:
        if processor_type.allow_creation:
            print(processor_type.type_)

In [ ]:
fetch_processor_types_sample(project_id, location)

In [ ]:
# read in excel file with 
data = pd.read_excel("CPTtoDesc.xlsx")

In [20]:
data.head()

,CPT,Description
0,86152,Cell enumeration &id
1,86153,Cell enumeration phys interp
2,86890,Autologous blood process
3,86891,Autologous blood op salvage
4,86927,Plasma fresh frozen


Keep this in mind in case 

In [21]:
# sort data on CPT value
data = data.sort_values(by='CPT', ascending=True)

TypeError: '<' not supported between instances of 'str' and 'int'

In [30]:
data["Description"] = data["Description"].apply(lambda x: (re.sub(r'[^\w\s]', '', x)))


In [32]:
# make all descriptions lowercase
data["Description"] = data["Description"].apply(lambda x: x.lower())

In [38]:
def desc2CPT(inputDesc: str) -> str:
    # make input lowercase
    inputDesc = inputDesc.lower()
    # remove punctuation
    inputDesc = re.sub(r'[^\w\s]', '', inputDesc)
    # vectorize the input
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data["Description"])
    inputVec = vectorizer.transform([inputDesc])
    # calculate cosine similarity
    cosineSimilarities = cosine_similarity(inputVec, X)
    # get the index of the most similar description
    index = cosineSimilarities.argmax()
    # get the CPT code of the most similar description
    CPT = data.iloc[index]["CPT"]
    return CPT


In [41]:
desc2CPT("Mri of chest")

71551